# 使用dropout正则化解决拟合问题

（1）导入需要的包和数据集

In [1]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
import numpy  as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

（2）获得数据对其标准化

In [2]:
# 载入数据
boston = load_boston()
X,y   = (boston.data, boston.target)
dim = X.shape[1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
num_train = X_train.shape[0]
#对训练数据进行标准化
mean=X_train.mean(axis=0)
std=X_train.std(axis=0)

In [3]:
X_train-=mean
X_train/=std

X_test-=mean
X_test/=std

In [4]:
# 转成tensor
train_data=torch.from_numpy(X_train)
# 查看数据
dtype = torch.FloatTensor
train_data.type(dtype)

tensor([[-0.3726, -0.4996, -0.7049,  ..., -0.4846,  0.3717, -0.4110],
        [-0.3971, -0.4996, -0.0449,  ...,  0.3365,  0.2050, -0.3877],
        [-0.4027,  0.7712, -0.8868,  ..., -0.8496,  0.3666, -0.1819],
        ...,
        [-0.3981, -0.4996, -0.1594,  ..., -0.3022,  0.4034, -0.3301],
        [-0.3884, -0.4996, -0.6033,  ..., -0.2565,  0.3834,  0.8359],
        [-0.3995, -0.4996, -1.0128,  ..., -0.8496,  0.4304,  0.2721]])

In [5]:
#分别获得训练集和测试集
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
train_data=torch.from_numpy(X_train).float()
train_target=torch.from_numpy(y_train).float()
test_data=torch.from_numpy(X_test).float()
test_target=torch.from_numpy(y_test).float()

（3）构建模型

In [6]:
# 构建模型
# 普通
net1_overfitting = torch.nn.Sequential(
    torch.nn.Linear(13, 16),
    torch.nn.ReLU(),
    torch.nn.Linear(16, 32),
    torch.nn.ReLU(),
    torch.nn.Linear(32, 1),
)

# 正则化
net1_dropped = torch.nn.Sequential(
    torch.nn.Linear(13, 16),
    torch.nn.Dropout(0.5),  # drop 50% of the neuron
    torch.nn.ReLU(),
    torch.nn.Linear(16, 32),
    torch.nn.Dropout(0.5),  # drop 50% of the neuron
    torch.nn.ReLU(),
    torch.nn.Linear(32, 1),
)

（4）定义loss值选择迭代器

In [7]:
loss_func = torch.nn.MSELoss()
optimizer_ofit = torch.optim.Adam(net1_overfitting.parameters(), lr=0.01)
optimizer_drop = torch.optim.Adam(net1_dropped.parameters(), lr=0.01)

（5）训练可视化数据

In [8]:
from tensorboardX import SummaryWriter
writer = SummaryWriter(log_dir='logs')
for epoch in range(100):
    # 调用训练
    net1_overfitting.train()
    net1_dropped.train()
    # 获得预测值
    pred_ofit=  net1_overfitting(train_data)
    pred_drop = net1_dropped(train_data)    
    # 计算loss
    loss_ofit = loss_func(pred_ofit, train_target)
    loss_drop = loss_func(pred_drop, train_target)
    #计算梯度
    optimizer_ofit.zero_grad()
    optimizer_drop.zero_grad()
    loss_ofit.backward()
    loss_drop.backward()
    optimizer_ofit.step()
    optimizer_drop.step()
    # 保存loss的数据与epoch数值
    writer.add_scalars('train_group_loss',{'origloss':loss_ofit.item(),'droploss':loss_drop.item()}, epoch)

    #进行预测
    net1_overfitting.eval()
    net1_dropped.eval() 
    test_pred_orig = net1_overfitting(test_data)
    test_pred_drop = net1_dropped(test_data)
    orig_loss=loss_func(test_pred_orig, test_target)
    drop_loss=loss_func(test_pred_drop, test_target)
    writer.add_scalars('test_group_loss',{'origloss':orig_loss.item(),'droploss':drop_loss.item()}, epoch)


d:\studyapp\anaconda\envs\gluon\lib\site-packages\torch\nn\modules\loss.py:432: UserWarning: Using a target size (torch.Size([404])) that is different to the input size (torch.Size([404, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
d:\studyapp\anaconda\envs\gluon\lib\site-packages\torch\nn\modules\loss.py:432: UserWarning: Using a target size (torch.Size([102])) that is different to the input size (torch.Size([102, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


# 批量正则化

In [1]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
import numpy  as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

In [2]:
# 载入数据
boston = load_boston()
X,y   = (boston.data, boston.target)
dim = X.shape[1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
num_train = X_train.shape[0]
#对训练数据进行标准化
mean=X_train.mean(axis=0)
std=X_train.std(axis=0)

In [3]:
X_train-=mean
X_train/=std

X_test-=mean
X_test/=std

In [4]:
#分别获得训练集和测试集
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
train_data=torch.from_numpy(X_train).float()
train_target=torch.from_numpy(y_train).float()
test_data=torch.from_numpy(X_test).float()
test_target=torch.from_numpy(y_test).float()

In [6]:
# 普通
net1_overfitting = torch.nn.Sequential(
    torch.nn.Linear(13, 16),
    torch.nn.ReLU(),
    torch.nn.Linear(16, 32),
    torch.nn.ReLU(),
    torch.nn.Linear(32, 1),
)
# 标准化
net1_nb = torch.nn.Sequential(
    torch.nn.Linear(13, 8),
    nn.BatchNorm1d(num_features=8),
    torch.nn.ReLU(),
    torch.nn.Linear(8, 4),
    nn.BatchNorm1d(num_features=4),  
    torch.nn.ReLU(),
    torch.nn.Linear(4, 1),
)
loss_func = torch.nn.MSELoss()
optimizer_ofit = torch.optim.Adam(net1_overfitting.parameters(), lr=0.01)
optimizer_nb = torch.optim.Adam(net1_nb.parameters(), lr=0.01)

In [7]:
from tensorboardX import SummaryWriter
writer = SummaryWriter(log_dir='log')
for epoch in range(200):
    net1_overfitting.train()
    net1_nb.train()
    
    pred_ofit = net1_overfitting(train_data)
    pred_nb = net1_nb(train_data)
    
    loss_ofit = loss_func(pred_ofit, train_target)
    loss_nb = loss_func(pred_nb, train_target)
    
    optimizer_ofit.zero_grad()
    optimizer_nb.zero_grad()
    
    loss_ofit.backward()
    loss_nb.backward()
    
    optimizer_ofit.step()
    optimizer_nb.step()
    net1_overfitting.eval()
    net1_nb.eval() 
    test_pred_orig = net1_overfitting(test_data)
    test_pred_nb = net1_nb(test_data)
    orig_loss=loss_func(test_pred_orig, test_target)
    nb_loss=loss_func(test_pred_nb, test_target)
    # 保存loss的数据与epoch数值
    #writer.add_scalar('train_loss', loss_ofit, t)
    writer.add_scalars('train_group_loss',{'orig_loss':loss_ofit.item(),'nb_loss':loss_nb.item(),}, epoch)
    writer.add_scalars('test_nb_loss',{'orig_loss':orig_loss.item(),'nb_loss':nb_loss.item()}, epoch)

d:\studyapp\anaconda\envs\gluon\lib\site-packages\torch\nn\modules\loss.py:432: UserWarning: Using a target size (torch.Size([404])) that is different to the input size (torch.Size([404, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
d:\studyapp\anaconda\envs\gluon\lib\site-packages\torch\nn\modules\loss.py:432: UserWarning: Using a target size (torch.Size([102])) that is different to the input size (torch.Size([102, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
